In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/SIMPLIFIED_with_locations.csv', index_col=0)

In [3]:
df.groupby(['beg_loc', 'beg_loc_lat', 'beg_loc_long'])['Captives arrived at 1st port'].sum().reset_index().to_csv('Origin_Locations_Grouped.csv')

In [4]:
unique_locations  = set(df['beg_loc'].values.tolist() + df['purchase_loc'].values.tolist() + df['end_loc'].values.tolist())

In [5]:
location_df = pd.DataFrame(list(unique_locations), columns = ['Place'])

In [6]:
location_df[['Origin_Count', 'Purchase_Count', 'Destination_Count']] = 0

In [7]:
beg_df = df.groupby('beg_loc')['Voyage ID'].count()
purchase_df = df.groupby('purchase_loc')['Voyage ID'].count()
end_df = df.groupby('end_loc')['Voyage ID'].count()

In [8]:
for place in location_df['Place'].values.tolist():
    try:
        beg_count = beg_df[place]
    except KeyError:
        beg_count = 0

    try:
        pur_count = purchase_df[place]
    except KeyError:
        pur_count = 0

    try:
        end_count = end_df[place]
    except KeyError:
        end_count = 0
    
    location_df.loc[location_df.Place == place, 'Origin_Count'] = beg_count
    location_df.loc[location_df.Place == place, 'Purchase_Count'] = pur_count
    location_df.loc[location_df.Place == place, 'Destination_Count'] = end_count

In [9]:
location_df[['latitude', 'longitude']] = None

In [10]:
def find_lat_long(place):
    try:
        subset = df.loc[df.beg_loc == place]
        if len(subset) > 0:
            return [subset['beg_loc_lat'].values[0], subset['beg_loc_long'].values[0]]
    except:
        pass

    try:
        subset = df.loc[df.purchase_loc == place]
        if len(subset) > 0:
            return [subset['purchase_loc_lat'].values[0], subset['purchase_loc_long'].values[0]]
    except:
        pass

    try:
        subset = df.loc[df.end_loc == place]
        if len(subset) > 0:
            return [subset['end_loc_lat'].values[0], subset['end_loc_long'].values[0]]
    except:
        pass


    return [None, None]
     

In [11]:
location_df['latitude'] = location_df.Place.apply(lambda x: find_lat_long(x)[0])
location_df['longitude'] = location_df.Place.apply(lambda x: find_lat_long(x)[1])

In [12]:
location_df.isnull().sum()

Place                 0
Origin_Count          0
Purchase_Count        0
Destination_Count     0
latitude             15
longitude            15
dtype: int64

In [13]:
location_df.to_csv('Locations_Involved.csv')